In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In Qiskit werden Schaltkreise innerhalb der Transpiler-Stufen mittels eines DAG (Directed Acyclic Graph) dargestellt. Im Allgemeinen besteht ein DAG aus Knoten (auch "Vertices" genannt) und gerichteten Kanten, die Knotenpaare in einer bestimmten Orientierung verbinden. Diese Darstellung wird in `qiskit.dagcircuit.DAGCircuit`-Objekten gespeichert, die aus einzelnen `DagNode`-Objekten zusammengesetzt sind. Der Vorteil dieser Darstellung gegenüber einer reinen Liste von Gates (d.h. einer Netzliste) besteht darin, dass der Informationsfluss zwischen Operationen explizit ist, was Transformationsentscheidungen erleichtert.

Dieser Leitfaden demonstriert, wie Sie mit DAGs arbeiten und diese zur Entwicklung eigener Transpiler-Passes nutzen. Er beginnt mit dem Aufbau eines einfachen Schaltkreises und der Untersuchung seiner DAG-Darstellung, erkundet dann grundlegende DAG-Operationen und implementiert einen benutzerdefinierten `BasicMapper`-Pass.

## Einen Schaltkreis erstellen und seinen DAG untersuchen

Das folgende Code-Beispiel veranschaulicht den DAG durch die Erstellung eines einfachen Schaltkreises, der einen Bell-Zustand vorbereitet und eine $R_Z$-Rotation anwendet, abhängig vom Messergebnis.
{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
```
</details>

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.converters import circuit_to_dag
from qiskit.visualization import circuit_drawer
from qiskit.visualization.dag_visualization import dag_drawer

# Create circuit
q = QuantumRegister(3, "q")
c = ClassicalRegister(3, "c")
circ = QuantumCircuit(q, c)
circ.h(q[0])
circ.cx(q[0], q[1])
circ.measure(q[0], c[0])

# Qiskit 2.0 uses if_test instead of c_if
with circ.if_test((c, 2)):
    circ.rz(0.5, q[1])

circuit_drawer(circ, output="mpl")

<Image src="../docs/images/guides/DAG-representation/extracted-outputs/1d16892a-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/DAG-representation/extracted-outputs/1d16892a-0.svg)

Im DAG gibt es drei Arten von Graphknoten: Qubit-/Clbit-Eingabeknoten (grün), Operationsknoten (blau) und Ausgabeknoten (rot). Jede Kante zeigt den Datenfluss (oder die Abhängigkeit) zwischen zwei Knoten an. Verwenden Sie die Funktion qiskit.tools.visualization.dag_drawer(), um den DAG dieses Schaltkreises zu visualisieren. (Installieren Sie dazu die [Graphviz-Bibliothek](https://graphviz.org/download/).)

In [2]:
# Convert to DAG
dag = circuit_to_dag(circ)
dag_drawer(dag)

<Image src="../docs/images/guides/DAG-representation/extracted-outputs/e498faa3-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/DAG-representation/extracted-outputs/e498faa3-0.avif)

## Grundlegende DAG-Operationen
Die folgenden Code-Beispiele demonstrieren gängige Operationen mit DAGs, einschließlich des Zugriffs auf Knoten, des Hinzufügens von Operationen und des Ersetzens von Teilschaltkreisen. Diese Operationen bilden die Grundlage für den Aufbau von Transpiler-Passes.
## Alle Operationsknoten im DAG abrufen
Die Methode `op_nodes()` gibt eine iterierbare Liste von `DAGOpNode`-Objekten im Schaltkreis zurück:

In [3]:
dag.op_nodes()

[DAGOpNode(op=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qargs=(<Qubit register=(3, "q"), index=0>,), cargs=()),
 DAGOpNode(op=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qargs=(<Qubit register=(3, "q"), index=0>, <Qubit register=(3, "q"), index=1>), cargs=()),
 DAGOpNode(op=Instruction(name='measure', num_qubits=1, num_clbits=1, params=[]), qargs=(<Qubit register=(3, "q"), index=0>,), cargs=(<Clbit register=(3, "c"), index=0>,)),
 DAGOpNode(op=Instruction(name='if_else', num_qubits=1, num_clbits=3, params=[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f912f47db10>, None]), qargs=(<Qubit register=(3, "q"), index=1>,), cargs=(<Clbit register=(3, "c"), index=0>, <Clbit register=(3, "c"), index=1>, <Clbit register=(3, "c"), index=2>))]

Each node is an instance of the `DAGOpNode` class:

In [4]:
node = dag.op_nodes()[3]
print("node name:", node.name)
print("op:", node.op)
print("qargs:", node.qargs)
print("cargs:", node.cargs)
print("condition:", node.op.condition)

node name: if_else
op: Instruction(name='if_else', num_qubits=1, num_clbits=3, params=[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f912f4ceed0>, None])
qargs: (<Qubit register=(3, "q"), index=1>,)
cargs: (<Clbit register=(3, "c"), index=0>, <Clbit register=(3, "c"), index=1>, <Clbit register=(3, "c"), index=2>)
condition: (ClassicalRegister(3, 'c'), 2)


Jeder Knoten ist eine Instanz der Klasse `DAGOpNode`:

In [5]:
from qiskit.circuit.library import HGate

dag.apply_operation_back(HGate(), qargs=[q[0]])
dag_drawer(dag)

<Image src="../docs/images/guides/DAG-representation/extracted-outputs/3c144b49-0.avif" alt="Output of the previous code cell" />

## Add an Operation to the Front
An operation is added to the beginning of the DAGCircuit using the `apply_operation_front()` method. This inserts the specified gate before all existing operations in the circuit, effectively making it the first operation executed.

In [6]:
from qiskit.circuit.library import CCXGate

dag.apply_operation_front(CCXGate(), qargs=[q[0], q[1], q[2]])
dag_drawer(dag)

<Image src="../docs/images/guides/DAG-representation/extracted-outputs/ed80a69f-0.avif" alt="Output of the previous code cell" />

## Eine Operation am Ende hinzufügen
Eine Operation wird mit der Methode `apply_operation_back()` am Ende des DAGCircuit hinzugefügt. Dadurch wird das angegebene Gate so angefügt, dass es nach allen vorhandenen Operationen im Schaltkreis auf die angegebenen Qubits wirkt.

In [7]:
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit.library import CHGate, U2Gate, CXGate

# Build sub-DAG
mini_dag = DAGCircuit()
p = QuantumRegister(2, "p")
mini_dag.add_qreg(p)
mini_dag.apply_operation_back(CHGate(), qargs=[p[1], p[0]])
mini_dag.apply_operation_back(U2Gate(0.1, 0.2), qargs=[p[1]])

# Replace CX with mini_dag
cx_node = dag.op_nodes(op=CXGate).pop()
dag.substitute_node_with_dag(cx_node, mini_dag, wires=[p[0], p[1]])
dag_drawer(dag)

<Image src="../docs/images/guides/DAG-representation/extracted-outputs/fdb3dd70-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/DAG-representation/extracted-outputs/3c144b49-0.avif)

## Add an Operation to the Front
An operation is added to the beginning of the DAGCircuit using the `apply_operation_front()` method. This inserts the specified gate before all existing operations in the circuit, effectively making it the first operation executed.

In [8]:
from qiskit.converters import dag_to_circuit

new_circ = dag_to_circuit(dag)
circuit_drawer(new_circ, output="mpl")

<Image src="../docs/images/guides/DAG-representation/extracted-outputs/786571f7-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/DAG-representation/extracted-outputs/ed80a69f-0.avif)

## Substitute a node with a subcircuit
A node representing a specific operation in the DAGCircuit is replaced with a subcircuit. First, a new sub-DAG is constructed with the desired sequence of gates, then the target node is substituted by this sub-DAG using `substitute_node_with_dag()`, preserving the connections to the rest of the circuit.

In [9]:
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.transpiler import Layout
from qiskit.circuit.library import SwapGate


class BasicSwap(TransformationPass):
    def __init__(self, coupling_map, initial_layout=None):
        super().__init__()
        self.coupling_map = coupling_map
        self.initial_layout = initial_layout

    def run(self, dag):
        new_dag = DAGCircuit()
        for qreg in dag.qregs.values():
            new_dag.add_qreg(qreg)
        for creg in dag.cregs.values():
            new_dag.add_creg(creg)

        if self.initial_layout is None:
            self.initial_layout = Layout.generate_trivial_layout(
                *dag.qregs.values()
            )

        current_layout = self.initial_layout.copy()

        for layer in dag.serial_layers():
            subdag = layer["graph"]
            for gate in subdag.two_qubit_ops():
                q0, q1 = gate.qargs
                p0 = current_layout[q0]
                p1 = current_layout[q1]

                if self.coupling_map.distance(p0, p1) != 1:
                    path = self.coupling_map.shortest_undirected_path(p0, p1)
                    for i in range(len(path) - 2):
                        wire1, wire2 = path[i], path[i + 1]
                        qubit1 = current_layout[wire1]
                        qubit2 = current_layout[wire2]
                        new_dag.apply_operation_back(
                            SwapGate(), qargs=[qubit1, qubit2]
                        )
                        current_layout.swap(wire1, wire2)

            new_dag.compose(
                subdag, qubits=current_layout.reorder_bits(new_dag.qubits)
            )

        return new_dag

![Output of the previous code cell](../docs/images/guides/DAG-representation/extracted-outputs/fdb3dd70-0.avif)

After all transformations are completed, the DAG can be converted back into a regular `QuantumCircuit` object. This is how the transpiler pipeline operates. A circuit is taken, processed in DAG form, and a transformed circuit is produced as output.

In [10]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit import QuantumRegister, QuantumCircuit

q = QuantumRegister(7, "q")
in_circ = QuantumCircuit(q)
in_circ.h(q[0])
in_circ.cx(q[0], q[4])
in_circ.cx(q[2], q[3])
in_circ.cx(q[6], q[1])
in_circ.cx(q[5], q[0])
in_circ.rz(0.1, q[2])
in_circ.cx(q[5], q[0])

coupling = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6]]
coupling_map = CouplingMap(couplinglist=coupling)

pm = PassManager()
pm.append(BasicSwap(coupling_map))

out_circ = pm.run(in_circ)

in_circ.draw(output="mpl")
out_circ.draw(output="mpl")

<Image src="../docs/images/guides/DAG-representation/extracted-outputs/2f35375f-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/DAG-representation/extracted-outputs/786571f7-0.svg)

## Implement a BasicMapper pass
The DAG structure can be leveraged for writing transpiler passes. In the example below, a `BasicMapper` pass is implemented to map an arbitrary circuit onto a device with restricted qubit connectivity. For additional guidance, refer to the guide on [writing custom transpiler passes](/guides/custom-transpiler-pass).

The pass is defined as a `TransformationPass`, meaning that it modifies the circuit. It does so by traversing the DAG layer-by-layer, checking whether each instruction satisfies the constraints imposed by the device's coupling map. If a violation is detected, a swap path is determined and the necessary SWAP gates are inserted accordingly.

When creating a transpiler pass, the first decision involves selecting whether the pass should inherit from `TransformationPass` or `AnalysisPass`. Transformation passes are designed to modify the circuit, whereas analysis passes are intended only to extract information for use by subsequent passes. The main functionality is then implemented in the `run(dag)` method. Finally, the pass should be registered within the `qiskit.transpiler.passes` module.

In this specific pass, the DAG is traversed layer-by-layer (where each layer contains operations that act on disjoint sets of qubits and can thus be executed independently). For each operation, if the coupling map constraints are not met, a suitable swap path is identified, and the required swaps are inserted to bring the involved qubits into adjacency.